In [5]:
# Import all necessary modules
import sys
# sys.path.append('/home1/esolo/ptsa_new/')
# sys.path.append('/home1/mran/PTSA_GIT/')
import numpy.testing
import numpy as np
# from ptsa.data.readers.IndexReader import JsonIndexReader # 08/30/17: Added for json file format
from ptsa.data.readers import BaseEventReader
# from ptsa.data.readers.TalReader import TalReader
from ptsa.data.readers import EEGReader
from ptsa.data.filters import MonopolarToBipolarMapper
from ptsa.data.filters import ButterworthFilter
# from ptsa.data.filters.MorletWaveletFilter import MorletWaveletFilter
from ptsa.data.filters import ResampleFilter
# from ptsa.data.common import xr
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats.mstats import zscore
from time import time
start_time = time()

#Added modules 
from pylab import *

In [ ]:
# Find where site packages are located
# import site; site.getsitepackages()

# Unix qlogin command:
# qlogin -l h_vmem=8G

In [ ]:
# from Electrophysiology_Functions import * # This will import the python file of the electrophysiology functions (NOT ALL FUNCTIONS PRESENT)

In [ ]:
# Import plotting modules
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import cPickle as pickle
import os.path

In [ ]:
# FUNCTION OUTDATED: No longer use .mat files
def get_all_subs_math(taskname, rhino_prefix='/'):
    from glob import glob
    
    fns = glob(rhino_prefix+'/data/events/'+taskname+'/*_math.mat')
    subs = [s.split('/')[-1].split('_m')[0] for s in fns]
    
    return(subs)

In [ ]:
def ReadSubjBaseEvents_Json(study_name, subject):
    ### Read in subjects events and electrode positions from the Json files 
    ### Updated: 8/25/17 - technically functional, poorly written
  
    # Work around for study_name changes from my old code to json experiment names
    if study_name == 'RAM_FR1':
        experiment = 'FR1'
    elif study_name == 'RAM_CatFR1':
        experiment = 'catFR1'
    
    if study_name == 'pyFR':
        # Old pathway with .mat files
        e_path = '/data/events/'+study_name+'/'+subject+'_math.mat'
        tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat'

        # ------------------- READING EVENTS
        base_e_reader = BaseEventReader(filename=e_path, eliminate_events_with_no_eeg=True)
        base_events = base_e_reader.read()
        base_events = base_events[base_events.type == 'PROB']

        # ------------------- READING TAL STRUCTS
        tal_reader = TalReader(filename=tal_path)
        monopolar_channels = tal_reader.get_monopolar_channels()
        bipolar_pairs = tal_reader.get_bipolar_pairs()

    elif study_name == 'RAM_FR1_CatFR1':
        if subject[-2] == '_': # Detects if subject has multiple montages
            s_montage = subject # Keeps the montage number on the subject name i.e. '_1'
            montage = subject[-1] # Delineate which montage will be used for this subject
            subject = subject[:-2] # Redefine subject without montage specified
            
            ## Loads all events and then restricts to the specific montage
            jr = JsonIndexReader('/protocols/r1.json')
            
            ## RAM_FR1 
            experiment ='FR1'
            basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment))) #Sorted to organize sessions in ascending order
            base_events = []
            for indx, session_path in enumerate(basefile_path): ## TEMP FIX: List order reversed to be in numerical session order
                base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                base_events.append(base_e_reader.read())

            base_events = np.concatenate(base_events).view(np.recarray)
            base_events = base_events[base_events.type == 'PROB']

            event_montages = [int(x[2]) for x in base_events.montage] # Extracts montage numbers from the events
            subject_montage = (np.ones(len(event_montages)) * int(montage)) # Creates an array for boolean comparison to the event montage
            FR1_base_events = base_events[np.array(event_montages) == np.array(subject_montage)]

            ## RAM_CatFR1 
            experiment ='catFR1'
            basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment)))
            base_events = []
            for indx, session_path in enumerate(basefile_path): ## TEMP FIX: List order reversed to be in numerical session order [::-1] (was the fix, may be the problem)
                base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                base_events.append(base_e_reader.read())

            base_events = np.concatenate(base_events).view(np.recarray)
            base_events = base_events[base_events.type == 'PROB']

            event_montages = [int(x[2]) for x in base_events.montage] # Extracts montage numbers from the events
            subject_montage = (np.ones(len(event_montages)) * int(montage)) # Creates an array for boolean comparison to the event montage
            CatFR1_base_events = base_events[np.array(event_montages) == np.array(subject_montage)]

            # Making the CatFR1 sessions have unique numbering (added as additional session to FR1)
            CatFR1_base_events.session = CatFR1_base_events.session + max(np.unique(FR1_base_events.session))+1

            # Combining the base_events where subjects completed both FR1 and CatFR1
            base_events = np.hstack((FR1_base_events,CatFR1_base_events))

            ## Loads specific electrode montage - for new json format
#             pairs = list(jr.aggregate_values('pairs',subject=subject,experiment=experiment))[::-1]
#             montage_pair = [int(x[55]) for x in pairs]
#             from itertools import compress
#             tal_path = list(compress(pairs,montage_pair == (np.ones(len(montage_pair)) * int(montage))))[0]
            
            # Loading tal_struct from old .mat files
            if int(montage) != 0:
                # Loads tal_path with montage specified
                tal_path = '/data/eeg/'+s_montage+'/tal/'+s_montage+'_talLocs_database_bipol.mat' # Old tal_struct loading
            else:
                # Corrects since subjects are not labeled '_0'
                tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat' # Old tal_struct loading

            # Reading Tal Structs
            tal_reader = TalReader(filename=tal_path)
            monopolar_channels = tal_reader.get_monopolar_channels()
            bipolar_pairs = tal_reader.get_bipolar_pairs()
        
        else:
            # Reads in Json protocol file to extract file paths
            jr = JsonIndexReader('/protocols/r1.json')

            ## RAM_FR1
            experiment = 'FR1'
            FR1_basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment)))
            FR1_base_events = []
            for indx, session_path in enumerate(FR1_basefile_path): ## TEMP FIX: List order reversed to be in numerical session order
                base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                FR1_base_events.append(base_e_reader.read())
            FR1_base_events = np.concatenate(FR1_base_events).view(np.recarray)
            FR1_base_events = FR1_base_events[FR1_base_events.type == 'PROB']

            ## RAM_CatFR1
            experiment = 'catFR1'
            CatFR1_basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment)))
            CatFR1_base_events = []
            for indx, session_path in enumerate(CatFR1_basefile_path): ## TEMP FIX: List order reversed to be in numerical session order
                base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                CatFR1_base_events.append(base_e_reader.read())
            CatFR1_base_events = np.concatenate(CatFR1_base_events).view(np.recarray)
            CatFR1_base_events = CatFR1_base_events[CatFR1_base_events.type == 'PROB']

            # Making the CatFR1 sessions have unique numbering (added as additional session to FR1)
            CatFR1_base_events.session = CatFR1_base_events.session + max(np.unique(FR1_base_events.session))+1

            # Combining the base_events where subjects completed both FR1 and CatFR1
            base_events = np.hstack((FR1_base_events,CatFR1_base_events))

            ## Tal Path
#             tal_path = jr.get_value('pairs',subject=subject,experiment=experiment) ## New .json format path for loading tal_struct - may need to use in the future but doesn't work now
            #### Recovers tal_path for old .mat files using the montage number from json file
            # Below finds if the montage is anything other than 0
            pair = jr.get_value('pairs',subject=subject,experiment=experiment)
            montage_pair = int(pair[55])
            if montage_pair != 0:
                subject = subject+'_'+str(montage_pair)
            tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat' # Old tal_struct loading path

            tal_reader = TalReader(filename=tal_path)
            monopolar_channels = tal_reader.get_monopolar_channels()
            bipolar_pairs = tal_reader.get_bipolar_pairs()
    
    else: 
        ## Loads data for specific subject with multiple montages
        if subject[-2] == '_': # Detects if subject has multiple montages
            s_montage = subject # Keeps the montage number on the subject name i.e. '_1'
            montage = subject[-1] # Delineate which montage will be used for this subject
            subject = subject[:-2] # Redefine subject without montage specified
            
            ## Loads all events and then restricts to the specific montage
            jr = JsonIndexReader('/protocols/r1.json')
            basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment)))
            base_events = []
            for indx, session_path in enumerate(basefile_path): ## TEMP FIX: List order reversed to be in numerical session order
                base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                base_events.append(base_e_reader.read())

            base_events = np.concatenate(base_events).view(np.recarray)
            base_events = base_events[base_events.type == 'PROB']
            
            event_montages = [int(x[2]) for x in base_events.montage] # Extracts montage numbers from the events
            subject_montage = (np.ones(len(event_montages)) * int(montage)) # Creates an array for boolean comparison to the event montage
            base_events = base_events[np.array(event_montages) == np.array(subject_montage)]
                
            ## Loads specific electrode montage - only new for new json format loading
#             pairs = list(jr.aggregate_values('pairs',subject=subject,experiment=experiment))[::-1]
#             montage_pair = [int(x[55]) for x in pairs]
#             from itertools import compress
#             tal_path = list(compress(pairs,montage_pair == (np.ones(len(montage_pair)) * int(montage))))[0]
            
            # Loading tal_struct from old .mat files
            if int(montage) != 0:
                # Loads tal_path with montage specified
                tal_path = '/data/eeg/'+s_montage+'/tal/'+s_montage+'_talLocs_database_bipol.mat' # Old tal_struct loading
            else:
                # Corrects since subjects are not labeled '_0'
                tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat' # Old tal_struct loading
  
            # Reading Tal Structs
            tal_reader = TalReader(filename=tal_path)
            monopolar_channels = tal_reader.get_monopolar_channels()
            bipolar_pairs = tal_reader.get_bipolar_pairs()
        else: 
            ## Regular pathway
            try: 
                # Reads in Json protocol file to extract file paths
                jr = JsonIndexReader('/protocols/r1.json')
                # subject = jr.subjects(experiment='FR1')[12] # Way to read in subject manually (DON'T USE)

                # Similar to e_path
                basefile_path = list(sorted(jr.aggregate_values('math_events',subject=subject,experiment=experiment)))
#                 tal_path = jr.get_value('pairs',subject=subject,experiment=experiment) # New json format, not using currently
                
                # Recovers old tal_path, making sure montage number is correct 
                pair = jr.get_value('pairs',subject=subject,experiment=experiment)
                montage_pair = int(pair[55])
                if montage_pair != 0:
                    subject = subject+'_'+str(montage_pair)
                tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat' # Old tal_struct loading path

                base_events = []
                for indx, session_path in enumerate(basefile_path): ## TEMP FIX: List order reversed to be in numerical session order
                    base_e_reader = BaseEventReader(filename=session_path, eliminate_events_with_no_eeg=True)
                    base_events.append(base_e_reader.read())

                base_events = np.concatenate(base_events).view(np.recarray) # Converting back from list to perform recarray operations
                base_events = base_events[base_events.type == 'PROB']

                # Reading Tal Structs
                tal_reader = TalReader(filename=tal_path)
                monopolar_channels = tal_reader.get_monopolar_channels()
                bipolar_pairs = tal_reader.get_bipolar_pairs()
            except:
                ### Attempts to find JSON format and then tries old method
                e_path = '/data/events/'+study_name+'/'+subject+'_math.mat'
                tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat'

                # ------------------- READING EVENTS
                base_e_reader = BaseEventReader(filename=e_path, eliminate_events_with_no_eeg=True)
                base_events = base_e_reader.read()
                base_events = base_events[base_events.type == 'PROB']

                # ------------------- READING TAL STRUCTS
                tal_reader = TalReader(filename=tal_path)
                monopolar_channels = tal_reader.get_monopolar_channels()
                bipolar_pairs = tal_reader.get_bipolar_pairs()
    # Changing to recarray
    base_events = base_events.view(np.recarray)
    
    return base_events, tal_reader, bipolar_pairs, monopolar_channels

In [1]:
def read_subj_base_events_elec(study_name, subject):
    # Read in subjects events and electrode positions
    
    if study_name == 'RAM_FR1_CatFR1':
        
        # ------------------- READING EVENTS
        # RAM_FR1
        e_path = '/data/events/RAM_FR1/'+subject+'_math.mat'
        base_e_reader = BaseEventReader(filename=e_path, eliminate_events_with_no_eeg=True)
        FR1_base_events = base_e_reader.read()
        FR1_base_events = FR1_base_events[FR1_base_events.type == 'PROB']
        
        # RAM_CatFR1
        e_path = '/data/events/RAM_CatFR1/'+subject+'_math.mat'
        base_e_reader = BaseEventReader(filename=e_path, eliminate_events_with_no_eeg=True)
        CatFR1_base_events = base_e_reader.read()
        CatFR1_base_events = CatFR1_base_events[CatFR1_base_events.type == 'PROB']
        
        # Making the CatFR1 sessions have unique numbering (added as additional session to FR1)
        CatFR1_base_events.session = CatFR1_base_events.session + max(np.unique(FR1_base_events.session))+1

        # Combining the base_events where subjects completed both FR1 and CatFR1
        base_events = np.hstack((FR1_base_events,CatFR1_base_events))
        
        # ------------------- READING TAL STRUCTS
        tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat'
        tal_reader = TalReader(filename=tal_path)
        monopolar_channels = tal_reader.get_monopolar_channels()
        bipolar_pairs = tal_reader.get_bipolar_pairs()
           
    else:
        # Regular pathway
        e_path = '/data/events/'+study_name+'/'+subject+'_math.mat'
        tal_path = '/data/eeg/'+subject+'/tal/'+subject+'_talLocs_database_bipol.mat'

        # ------------------- READING EVENTS
        base_e_reader = BaseEventReader(filename=e_path, eliminate_events_with_no_eeg=True)
        base_events = base_e_reader.read()
        base_events = base_events[base_events.type == 'PROB']

        # ------------------- READING TAL STRUCTS
        tal_reader = TalReader(filename=tal_path)
        monopolar_channels = tal_reader.get_monopolar_channels()
        bipolar_pairs = tal_reader.get_bipolar_pairs()
        
    # Changing to recarray
    base_events = base_events.view(np.recarray)
    
    return base_events, tal_reader, bipolar_pairs, monopolar_channels

In [ ]:
def load_eeg_data(base_events,monopolar_channels):
    # Reading in events with the response time as the end time in seconds
    data = [] # intialize the list where to store each event object

    for i in range(0,base_events.shape[0]):

        eeg_reader = EEGReader(events=base_events[i:i+1], channels=monopolar_channels,
                               start_time=0.0, end_time=float(base_events[i]['rectime'])/1000, buffer_time=1.0)

        base_eegs = eeg_reader.read() # temporary reading of events
        data.append(base_eegs) # list of event objects 
    return data

In [ ]:
def wavelet_function(data, freqs,bipolar_pairs):
    # Specify event or range of events
    data_pow_wavelet = []
    #freqs=np.logspace(np.log10(3), np.log10(180), 8)
    
    for i_event in range(0,len(data)):

        # Creating bipolar EEGs 
        m2b = MonopolarToBipolarMapper(time_series=data[i_event], bipolar_pairs=bipolar_pairs)
        bp_eegs = m2b.filter()

        # Filtering out line noise
        b_filter = ButterworthFilter(time_series=bp_eegs, freq_range=[58., 62.], filt_type='stop', order=4)
        bp_eegs_filtered = b_filter.filter()

        # Perform the Morlet Wavelet
        wf = MorletWaveletFilter(time_series=bp_eegs_filtered,
                             freqs=freqs,
                             output='power'                   
                             )
        pow_wavelet, phase_wavelet = wf.filter()

        # Remove the buffer time period
        pow_wavelet = pow_wavelet.remove_buffer(duration=1.0)
        np.log10(pow_wavelet.data, out=pow_wavelet.data); # due to power decay in high freq

        # Re-organize data structure and get rid of event dimension, since it is always 1
        pow_wavelet = pow_wavelet.transpose("events","bipolar_pairs","frequency","time") # Transpose for subsequent analysis
        pow_wavelet = np.squeeze(pow_wavelet) # Eliminating event dimension
        
        # Adding the data to a wavelet power structure
        data_pow_wavelet.append(pow_wavelet)
    return data_pow_wavelet

In [ ]:
def wavelet_downsample(data_pow_wavelet,downsampled_freq):
    # Down-sampling after the wavelet transform
#     downsampled_freq = 100 # If need to hard-code downsampled frequency

    data_pow_wavelet_ds = []
    # Loops through all events and downsamples to input frequency
    for i_event in range(0,len(data_pow_wavelet)):
        
        # Old version of downsampling
        # data_pow_wavelet_temp_ds = data_pow_wavelet[i_event].resampled(downsampled_freq) # Downsampling each event 
        # Updated version of downsamping
        resample_filter = ResampleFilter(time_series=data_pow_wavelet[i_event], resamplerate=100.0)
        data_pow_wavelet_temp_ds = resample_filter.filter()
        
        # IMPORTANT: Adds event data to attributes
        data_pow_wavelet_temp_ds.attrs['events'] = data_pow_wavelet[i_event].events
        
        data_pow_wavelet_temp_ds.coords['samplerate'] = downsampled_freq # Adjusting samplerate to downsampled frequency
        data_pow_wavelet_ds.append(data_pow_wavelet_temp_ds)      
    return data_pow_wavelet_ds

In [ ]:
def z_score_function(data_pow_wavelet):
    ## Calculating the z score
    mean_powers_by_event = []
    downsampled_freq = 100

    # Averaging across the time dimension
    for j_events in range(0,len(data_pow_wavelet)):
        temp_mean_powers = np.array(data_pow_wavelet[j_events].mean(skipna=True,axis=2)) # Averages across time dimension (last dimension)
        mean_powers_by_event.append(temp_mean_powers) # (events, electrode pairs, frequency bands)

    # Mean across events
    mean_powers = np.nanmean(mean_powers_by_event,axis=0) # mean power across events, now have (electrode pairs, frequency)
    std_powers = np.nanstd(mean_powers_by_event,axis=0) # Standard deviations of the means of each event (electrode pairs, frequency)

    # Re-creating the matrix with time and events
    data_mean_pow = []
    data_std_pow = []
    for k_events in range(0,len(data_pow_wavelet)):
        mean_pow_w_time = np.tile(mean_powers,[len(data_pow_wavelet[k_events].time),1,1]) # tiles along time dimension
        mean_pow_w_time = np.transpose(mean_pow_w_time,(1,2,0)) # re-arrange to dimensions in pow_wavelet (electrode pairs, frequency, time)
        data_mean_pow.append(mean_pow_w_time) # Adds tiled matrices to events structure

        std_pow_w_time = np.tile(std_powers,[len(data_pow_wavelet[k_events].time),1,1]) # tiles along time dimension for std
        std_pow_w_time = np.transpose(std_pow_w_time,(1,2,0)) # re-arrange to dimensions in pow_wavelet (electrode pairs, frequency, time)
        data_std_pow.append(std_pow_w_time) # Adds tiled matrices to events structure

    # Z score calculation
    data_z_pow_wavelet = [] # Comment out if you want to add more events (more than one session)

    for i_event in range(0,len(data_pow_wavelet)):
        z_pow_wavelet = (data_pow_wavelet[i_event] - data_mean_pow[i_event])/data_std_pow[i_event] # Z score equation
#         z_pow_wavelet = z_pow_wavelet.resampled(downsampled_freq) # Downsampling each event 
        data_z_pow_wavelet.append(z_pow_wavelet) # Blocks of (electrode pairs, frequency, time) for each event
    
    return data_z_pow_wavelet

In [ ]:
def z_score_function_updated(data_pow_wavelet):
    import copy
    ## Calculating the z score
    mean_powers_by_event = []
    data_z_pow_wavelet = [] # Comment out if you want to add more events (more than one session)

    downsampled_freq = 100

    # Concatenates all events together to set up calculation of mean and standard deviation below
    for j_events in range(0,len(data_pow_wavelet)):
        if j_events == 0:
            data_complt_session = data_pow_wavelet[0]
        else:
            data_complt_session = np.concatenate((data_complt_session,data_pow_wavelet[j_events]),axis=2) 
        
    # Mean across time
    mean_powers = np.nanmean(data_complt_session,axis=2) # mean power across events, now have (electrode pairs, frequency)
    std_powers = np.nanstd(data_complt_session,axis=2) # Standard deviations of the means of each event (electrode pairs, frequency)

    # Re-creating the matrix with time and events
    data_mean_pow = []
    data_std_pow = []
    for k_events in range(0,len(data_pow_wavelet)):
        mean_pow_w_time = np.tile(mean_powers,[len(data_pow_wavelet[k_events].time),1,1]) # tiles along time dimension
        mean_pow_w_time = np.transpose(mean_pow_w_time,(1,2,0)) # re-arrange to dimensions in pow_wavelet (electrode pairs, frequency, time)
        data_mean_pow.append(mean_pow_w_time) # Adds tiled matrices to events structure

        std_pow_w_time = np.tile(std_powers,[len(data_pow_wavelet[k_events].time),1,1]) # tiles along time dimension for std
        std_pow_w_time = np.transpose(std_pow_w_time,(1,2,0)) # re-arrange to dimensions in pow_wavelet (electrode pairs, frequency, time)
        data_std_pow.append(std_pow_w_time) # Adds tiled matrices to events structure

    # Z score calculation

    for i_event in range(0,len(data_pow_wavelet)):
        z_pow_wavelet = (data_pow_wavelet[i_event] - data_mean_pow[i_event])/data_std_pow[i_event] # Z score equation
        z_pow_wavelet.attrs = data_pow_wavelet[i_event].attrs # Keep attributes
        data_z_pow_wavelet.append(copy.deepcopy(z_pow_wavelet)) # Blocks of (electrode pairs, frequency, time) for each event
    
    return data_z_pow_wavelet

In [ ]:
def calculate_z_pow_wavelet(base_events,freqs,downsampled_freq,bipolar_pairs,monopolar_channels):
    # Loading EEG data, calculating Wavelet power, and performing z-score across multiple sessions
    #freqs=np.logspace(np.log10(3), np.log10(180), 8)
    # downsampled_freq= 100.0

    data_z_pow_wavelet = []

    # Finding unique sessions while preserving order
    uniq_indices = np.unique(base_events.session,return_index=True)[1]
    uniq_sessions = [base_events.session[index] for index in sorted(uniq_indices)]
    
#     for i_sessions in range(min(base_events['session']),max(base_events['session'])+1): # Analyzing all of the subject's sessions - OLD WAY
    for i_sessions in uniq_sessions: # Analyzing all of the subject's sessions
        base_events_filt = base_events[base_events['session'] == i_sessions] # Filtering for each subject session

        if len(base_events_filt) != 0:
            data_temp = load_eeg_data(base_events_filt,monopolar_channels) # Loading the EEG data for the session

            data_wavelet_temp = wavelet_function(data_temp,freqs,bipolar_pairs) # Performing the wavelet

            data_wavelet_temp_ds = wavelet_downsample(data_wavelet_temp,downsampled_freq) # Downsamples

            data_z_temp = z_score_function_updated(data_wavelet_temp_ds) # Z scores 

            data_z_pow_wavelet = data_z_pow_wavelet+data_z_temp
    return data_z_pow_wavelet

In [ ]:
def find_ROI_elec(tal_reader,ROI,cortex_side,study_name):
# Function finds all electrode in desired ROI and returns indices for analysis
    elec_index = []
        
    # Superior Frontal Gyrus
    if ROI == 'SFG':
        ROI_regions = ['superiorfrontal']
        
    # Middle Frontal Gyrus
    elif ROI == 'MFG':
        
        ROI_regions = ['caudalmiddlefrontal','rostralmiddlefrontal'] # regions defined by middle frontal gyrus
        
    # Inferior Frontal Gyrus
    elif ROI == 'IFG':
        ROI_regions = ['parsopercularis','parsorbitalis','parstriangularis'] # Regions defined by inferior frontal gyrus
    
    # Frontal Crotex (FC) # Added 2/2017
    elif ROI == 'FC':
        
        ROI_regions = ['superiorfrontal','caudalmiddlefrontal','rostralmiddlefrontal','parsopercularis','parsorbitalis','parstriangularis'] # Includes inferior, middle, superior regions
    
    # Occipital Cortex
    elif ROI == 'OC':
        ROI_regions = ['lateraloccipital','lingual','cuneus','pericalcarine']
    
    # Temporal Cortex
    elif ROI == 'TC':
        ROI_regions = ['superiortemporal','middletemporal','inferiortemporal'] # Previously included "Brodman area 37" but was not recognizing subjects, superiortemporal added 12/2017
        
    # Inferior Parietal Cortex
    elif ROI == 'IPC':
        ROI_regions = ['inferiorparietal','supramarginal']
        
    # Superior Parietal Cortex
    elif ROI == 'SPC':
        ROI_regions = ['superiorparietal','precuneus']
        
    # Parietal Cortex (PC) # Added 2/2017
    elif ROI == 'PC':
        ROI_regions = ['inferiorparietal','supramarginal','superiorparietal','precuneus']
        
    # Fusiform Gyrus
    elif ROI == 'FUS':
        ROI_regions = ['fusiform']
        
    ## Areas within parietal cortex separated # Added 12/2017
    elif ROI == 'IPG':
        ROI_regions = ['inferiorparietal']
    elif ROI == 'SPG':
        ROI_regions = ['superiorparietal']
    elif ROI == 'SMG':
        ROI_regions = ['supramarginal']
    elif ROI == 'PRC':
        ROI_regions = ['precuneus']
        
    ## Areas within the temporal cortex separated # Added 12/2017
    elif ROI == 'ITG':
        ROI_regions = ['inferiortemporal']     
    elif ROI == 'MTG':
        ROI_regions = ['middletemporal']
    elif ROI == 'STG':
        ROI_regions = ['superiortemporal']
    
    # Motor Cortex - Pre-Central Gyrus
    elif ROI == 'MC':
        ROI_regions = ['precentral']
        
    # Hippocampus (only for RAM subjects -- locTags) Subjects in pyFR have different function
    elif ROI == 'HIPP':
        ROI_regions = ['Left CA1','Left CA2','Left CA3','Left DG','Left Sub','Right CA1','Right CA2','Right CA3','Right DG','Right Sub']
        
    # Medial Temporal Lobe (only for RAM subjects -- locTags)
    elif ROI == 'MTL':
        ROI_regions = ['Left PRC','Right PRC','Left EC','Right EC','Left PHC','Right PHC']
        
        if ('locTag' in tal_reader.tal_struct_array.dtype.names == 'False') or (study_name == 'pyFR'): # If the location tags are not defined, uses Freesurfer segmentation
            ROI_regions = ['parahippocampal','entorhinal']
            ROI = 'MTL_noLocs' # Re-naming so code will not enter locTags if statement
    
    ## Locating all electrodes within the ROi regions defined above
    if (ROI == 'HIPP') and (study_name != 'pyFR') or (ROI == 'MTL') and (study_name != 'pyFR'):
        # These two ROIs are defined by the locTag structure
        for i_regions in ROI_regions:
            # Returns the indices of electrodes within the ROI
            elec_index_temp = np.where(tal_reader.tal_struct_array.locTag == i_regions)[0]

            # Adds current region's electrodes to previously identifed electrodes
            elec_index = np.hstack((elec_index,elec_index_temp))
    elif (ROI == 'HIPP') and (study_name == 'pyFR'):
        # pyFR study uses a different localization
        if study_name == 'pyFR':
            bipolar_pairs = tal_reader.get_bipolar_pairs()
            subject = tal_reader.tal_struct_array['subject'][0]
            elec_index = find_hip_elec_pyFR(subject,bipolar_pairs,tal_reader)
    else:
        # All regions other than the ones above use the avgSurf field for localization
        for i_regions in ROI_regions:
            # Returns the indices of electrodes within the ROI
            elec_index_temp = np.where(tal_reader.tal_struct_array.avgSurf['anatRegion'] == i_regions)[0]

            # Adds current region's electrodes to previously identifed electrodes
            elec_index = np.hstack((elec_index,elec_index_temp))
    
    elec_index = elec_index.astype('int') # Formatted to index arrays
    
    # Localizing electrodes to a particular side (i.e. Left or Right or Both)
    if cortex_side == 'Left':
        side_index = np.where(tal_reader.tal_struct_array.x < 0)[0] # Negative values in the x dimension indicate left cortex
        elec_index = np.intersect1d(side_index,elec_index) # Finding the overlap between the ROI and the cortex side
    elif cortex_side == 'Right':
        side_index = np.where(tal_reader.tal_struct_array.x > 0)[0] # Positive values in the x dimension indicate right cortex
        elec_index = np.intersect1d(side_index,elec_index) # Finding the overlap between the ROI and the cortex side
    
    return elec_index

In [ ]:
def find_hip_elec_pyFR(subject,bipolar_pairs,tal_reader):
# Locating hippocampal electrodes for pyFR
    
    # Loading Ethan's hippocampal electrode file
    import scipy.io
    mat_file = scipy.io.loadmat('/data6/scratch/esolo/pyFR_hippocampal_elecs.mat') # Loading array with electrodes within hippocampus
    hip_electrodes = squeeze(mat_file['he']) # Extracting the hippocampal electrode variable from the .mat file

    # Redefining the bipolar_pairs as an integer array in order to compare with hipp array
    bipolar_pairs_int = np.zeros([len(bipolar_pairs),2])
    for i_pairs in range(0,len(bipolar_pairs)):
        [a,b] = bipolar_pairs[i_pairs]
        elec_pair_temp = [int(a),int(b)]
        bipolar_pairs_int[i_pairs,:] = elec_pair_temp

    # Selecting hippocampal electrodes
    hip_subj_elec = hip_electrodes[hip_electrodes['subject'] == subject]

    # Finding the indices of hippocampal electrodes within the bipolar_pairs
    elec_index = np.zeros(len(hip_subj_elec))
    num_hip_pairs = 0
    for i_hipp_pair in hip_subj_elec['channel']:
        for i_elec_pair in range(0,len(bipolar_pairs)):
            if np.array_equal(bipolar_pairs_int[i_elec_pair,:],i_hipp_pair[0]):
                elec_index[num_hip_pairs] = i_elec_pair
                num_hip_pairs = num_hip_pairs+1
    
    elec_index = elec_index.astype(int) # convert to integers
    
    ## MADE OBSOLETE - KEPT FOR REFERENCE
#     # Filters the electrodes based on hemisphere 
#     if len(elec_index) != 0: 
#         elec_boolean = []
#         if cortex_side == 'Right':
#             for i_elec in elec_index:
#                 elec_boolean = elec_boolean + ([tal_reader.tal_struct_array[['Loc1']][i_elec][0] == 'Right Cerebrum'])
#             elec_index = elec_index[array(elec_boolean)]
#         elif cortex_side == 'Left':
#             for i_elec in elec_index:
#                 elec_boolean = elec_boolean + ([tal_reader.tal_struct_array[['Loc1']][i_elec][0] == 'Left Cerebrum'])
#             elec_index = elec_index[array(elec_boolean)]
    
    # print elec_index
    return elec_index

In [ ]:
def filter_correct(data_z_pow_wavelet,base_events):
    # Creating index for all correct events
    correct_index = np.where(base_events['iscorrect'] == 1)[0]

    # Creating an index for all incorrect events
    incorrect_index = np.where(base_events['iscorrect'] == 0)[0]

    # Only includes events that have a correct response
    data_z_pow_wavelet_cor = [data_z_pow_wavelet[i] for i in correct_index]

    # Only includes events that have an incorrect response
    data_z_pow_wavelet_incor = [data_z_pow_wavelet[i] for i in incorrect_index]
    
    # Only including correct events in base_events
    events_boolean_cor = base_events['iscorrect'] == 1
    base_events = base_events[events_boolean_cor]
    
    return data_z_pow_wavelet_cor, data_z_pow_wavelet_incor, base_events

In [ ]:
def separate_rectime(data_z_pow_wavelet,base_events):
# Creates arrays separated at the median by the response time

    # Initialize all variables
    rectimes_by_event = []
    data_z_pow_wavelet_fst = [];
    data_z_pow_wavelet_slw = [];

    # Calculates mean response time
#     rectimes_by_event_mean = base_events['rectime'] < mean(base_events['rectime']) # Alternate approach -- differing number of subjects
    rectimes_by_event_mean = base_events['rectime'] < median(base_events['rectime']) # Alternate approach -- differing number of subjects
    
#     for i_event in range(0,len(data_z_pow_wavelet)):
#         rectimes_by_event.append(data_z_pow_wavelet[i_event].attrs['events'].values['rectime'])
#     mean_rectime = mean(rectimes_by_event) # Changed to mean
#     rectimes_by_event_mean = rectimes_by_event < mean_rectime # 'True' means it is a fast event

    # Separating fast and slow
    for i_events in range(0,len(rectimes_by_event_mean)):
        if rectimes_by_event_mean[i_events]:
            data_z_pow_wavelet_fst.append(data_z_pow_wavelet[i_events]) # Finding all fast events
        else:
            data_z_pow_wavelet_slw.append(data_z_pow_wavelet[i_events]) # Finding all slow events
    
    return data_z_pow_wavelet_fst,data_z_pow_wavelet_slw

In [ ]:
def separate_rectime_model(data_z_pow_wavelet,base_events):
    ## Loading the model for trial separation
    filename = 'ECoG_Model.sav'
    num_features = 5
    lin_regression = pickle.load(open(filename, 'rb'))
    model_coef_total = lin_regression.coef_[0:num_features] # Rest of the features are subject-related

    ## Model Factors/ Features
    problem_set = base_events.test

    # Extracting digits from problem
    first_digit = array([x[0] for x in problem_set])
    second_digit = array([y[1] for y in problem_set])
    third_digit =  array([z[2] for z in problem_set])

    # Problem Sum
    problem_sum = array(map(sum,problem_set)) # Adds the digits in each tuple which are the (x,y,z) values

    # Odd/Even Sum
    odd_even_sum = (problem_sum % 2 == 0).astype(int) # If sum of problem equals zero when divided by two

    # Triplet
    triplet = (first_digit == second_digit) & (second_digit == third_digit).astype(int)

    # Repeated Digits
    repeated_doubles = (first_digit == second_digit) | (second_digit == third_digit) | (first_digit == third_digit).astype(int)
    repeated_doubles = repeated_doubles - triplet # Remove counting triplets twice

    # Two Digits adding to multiples of 10
    multiple_of_ten = ((first_digit+second_digit) % 10 == 0) | ((second_digit+third_digit) % 10 == 0) | ((first_digit+third_digit) % 10 == 0).astype(int)

    ## Concatenate all factors into array
    RT_features = np.transpose(np.vstack((problem_sum,odd_even_sum,triplet,repeated_doubles,multiple_of_ten)))

    ## Calculate predicted response times
    rectime_predict = np.matmul(RT_features,model_coef_total)

    ## Separate trial based on predicted rectime

    # Initialize all variables
    data_z_pow_wavelet_fst = [];
    data_z_pow_wavelet_slw = [];

    # Calculate residuals after model prediction
    rectime_residual = base_events['rectime'] - rectime_predict
    
    # Separate by mean
    rectimes_by_event_mean = rectime_residual < median(rectime_residual) # Alternate approach -- differing number of subjects

    # Separate into slow and fast datasets
    for i_events in range(0,len(rectimes_by_event_mean)):
        if rectimes_by_event_mean[i_events]:
            data_z_pow_wavelet_fst.append(data_z_pow_wavelet[i_events]) # Finding all fast events
        else:
            data_z_pow_wavelet_slw.append(data_z_pow_wavelet[i_events]) # Finding all slow events

    return data_z_pow_wavelet_fst,data_z_pow_wavelet_slw

In [ ]:
def separate_rectime_predict(data_z_pow_wavelet,base_events):
    ## Loading the model for trial separation
    filename = 'ECoG_Model.sav'
    num_features = 5
    lin_regression = pickle.load(open(filename, 'rb'))
    model_coef_total = lin_regression.coef_[0:num_features] # Rest of the features are subject-related

    ## Model Factors/ Features
    problem_set = base_events.test

    # Extracting digits from problem
    first_digit = array([x[0] for x in problem_set])
    second_digit = array([y[1] for y in problem_set])
    third_digit =  array([z[2] for z in problem_set])

    # Problem Sum
    problem_sum = array(map(sum,problem_set)) # Adds the digits in each tuple which are the (x,y,z) values

    # Odd/Even Sum
    odd_even_sum = (problem_sum % 2 == 0).astype(int) # If sum of problem equals zero when divided by two

    # Triplet
    triplet = (first_digit == second_digit) & (second_digit == third_digit).astype(int)

    # Repeated Digits
    repeated_doubles = (first_digit == second_digit) | (second_digit == third_digit) | (first_digit == third_digit).astype(int)
    repeated_doubles = repeated_doubles - triplet # Remove counting triplets twice

    # Two Digits adding to multiples of 10
    multiple_of_ten = ((first_digit+second_digit) % 10 == 0) | ((second_digit+third_digit) % 10 == 0) | ((first_digit+third_digit) % 10 == 0).astype(int)

    ## Concatenate all factors into array
    RT_features = np.transpose(np.vstack((problem_sum,odd_even_sum,triplet,repeated_doubles,multiple_of_ten)))

    ## Calculate predicted response times
    rectime_predict = np.matmul(RT_features,model_coef_total)

    ## Separate trial based on predicted rectime

    # Initialize all variables
    data_z_pow_wavelet_fst = [];
    data_z_pow_wavelet_slw = [];
    
    # Separate by mean
    rectimes_by_event_mean = rectime_predict < median(rectime_predict) # Alternate approach -- differing number of subjects

    # Separate into slow and fast datasets
    for i_events in range(0,len(rectimes_by_event_mean)):
        if rectimes_by_event_mean[i_events]:
            data_z_pow_wavelet_fst.append(data_z_pow_wavelet[i_events]) # Finding all fast events
        else:
            data_z_pow_wavelet_slw.append(data_z_pow_wavelet[i_events]) # Finding all slow events

    return data_z_pow_wavelet_fst,data_z_pow_wavelet_slw

In [ ]:
def avg_time_bins(data_z_pow_wavelet,elec_index,time_bins):
    # Averaging across time bins for each event
    # time_bins = 10
    samp_freq = np.array(data_z_pow_wavelet[0].samplerate) # given in Hz (should also work for downsampled data)
#     samp_freq = 100 # Switch if using downsampled data is used
    
    # Initializing array size
    time_bin_power = np.zeros((len(data_z_pow_wavelet),len(data_z_pow_wavelet[0].bipolar_pairs),len(data_z_pow_wavelet[0].frequency),time_bins))
    time_bin_power_std = np.zeros((len(data_z_pow_wavelet),len(data_z_pow_wavelet[0].bipolar_pairs),len(data_z_pow_wavelet[0].frequency),time_bins))

    for k_event in range(0,len(data_z_pow_wavelet)):   
        bin_interval = float(np.array(data_z_pow_wavelet[k_event].time[-1])/time_bins) # Setting the bin interval for each event
        for i_bins in range(0,time_bins):
            # Calculating the bin indexing for each iteration
            lower_bin = int(round(bin_interval*float(i_bins)*samp_freq))
            upper_bin = int(round(bin_interval*float(i_bins+1)*samp_freq))
            time_bin_power[k_event,:,:,i_bins] = nanmean(np.array(data_z_pow_wavelet[k_event][:,:,lower_bin:upper_bin]),axis=2)
            # Standardizes each event with time bins Structure: (events, electrode_pairs, frequency,time bins)
    
    # Average across selected electrodes 
    time_bin_power_avg_elec = time_bin_power[:,elec_index,:,:] # Reducing to only selected electrodes
    
    # Averages across electrodes if more than one electrode in ROI
    if len(elec_index) > 1:
        time_bin_power_avg_elec = nanmean(time_bin_power_avg_elec,axis=1) # Average across selected electrodes
    else: # Reducing the extra dimension if one electrode
        time_bin_power_avg_elec = np.squeeze(time_bin_power_avg_elec,axis=1)
        
        
    # Averages across events and calculates the standard deviation
    time_bin_power_avg = nanmean(time_bin_power_avg_elec,axis=0) # Average across events
#     time_bin_power_std_avg = std(time_bin_power_avg_elec,axis=0) # Standard deviation across events (not necessary since standard deviation will be calculated later)  
        
    return time_bin_power_avg, time_bin_power_avg_elec

In [ ]:
def calculate_tstat(timebin_pow_events_fst,timebin_pow_events_slw,time_bins):
# Calculates the t-statistic for each frequency band and time bin
    from scipy import stats # import for t test

    num_freqs = np.shape(timebin_pow_events_fst)[1]
    
    tscore_results = np.zeros([num_freqs,time_bins]) # Length of the array is freqs x timebins

    # Calculating the t statistic between fast and slow events for each subject
    for i_freq in range(0,num_freqs):
        for i_timebins in range(0,time_bins):
            tresults_temp = stats.ttest_ind(array(timebin_pow_events_fst)[:,i_freq,i_timebins],array(timebin_pow_events_slw)[:,i_freq,i_timebins])
            tscore_results[i_freq,i_timebins] = tresults_temp.statistic # Fills an array of t_statistic for each subject
                
    return tscore_results

In [ ]:
def calculate_tstat_ROI(data_z_pow_wavelet_fst,data_z_pow_wavelet_slw,freqs):
    # Calculates t-statistic for every frequency for each subject and ROI
    
    from scipy import stats # import for t test
    
    # Initializing variables
    ttest_ROI = np.empty((len(freqs)))
    data_zpow_fst = np.empty((len(data_z_pow_wavelet_fst),len(data_z_pow_wavelet_fst[0].bipolar_pairs),len(data_z_pow_wavelet_fst[0].frequency)),dtype=float) # Array [Events, bipolar_pairs, frequency]
    data_zpow_slw = np.empty((len(data_z_pow_wavelet_slw),len(data_z_pow_wavelet_slw[0].bipolar_pairs),len(data_z_pow_wavelet_slw[0].frequency)),dtype=float)

    # Averaging over time dimesnion and re-structuring to array with events as a dimension
    for i_events in range(0,len(data_z_pow_wavelet_fst)):
        data_zpow_fst[i_events,:,:] = data_z_pow_wavelet_fst[i_events].mean(skipna=True,axis=-1) # [Bipolar_pairs, frequency] adding to an array with events dimension
    for i_events in range(0,len(data_z_pow_wavelet_slw)):
        data_zpow_slw[i_events,:,:] = data_z_pow_wavelet_slw[i_events].mean(skipna=True,axis=-1) # [Bipolar_pairs, frequency] adding to an array with events dimension
    data_mean_zpow_fst = nanmean(data_zpow_fst[:,elec_index,:],axis=1) # Averaging ONLY bipolar pairs within the ROI for fast events
    data_mean_zpow_slw = nanmean(data_zpow_slw[:,elec_index,:],axis=1) # Above with slow events

    # ttest between events for each frequency band
    for i_freqs in range(0,len(data_z_pow_wavelet_fst[0].frequency)):
        ttest_result = stats.ttest_ind(data_mean_zpow_fst[:,i_freqs],data_mean_zpow_slw[:,i_freqs])
        ttest_ROI[i_freqs] = ttest_result.statistic    
        
    return ttest_ROI

In [ ]:
def calculate_tstat_elec(data_z_pow_wavelet_fst, data_z_pow_wavelet_slw,low_freq,high_freq):
############# UNFINISHED ##########
# Calculates the t-statistic for each electrode pair for cortical plots

    from scipy import stats # import for t test

    num_freqs = np.shape(timebin_pow_events_fst)[1]
    
    # Calculating average power
    
    
    tscore_results = np.zeros([num_freqs,time_bins]) # Length of the array is freqs x timebins

    # Calculating the t statistic between fast and slow events for each subject
    for i_freq in range(0,num_freqs):
        for i_timebins in range(0,time_bins):
            tresults_temp = stats.ttest_ind(array(timebin_pow_events_fst)[:,i_freq,i_timebins],array(timebin_pow_events_slw)[:,i_freq,i_timebins])
            tscore_results[i_freq,i_timebins] = tresults_temp.statistic # Fills an array of t_statistic for each subject
                
    return tscore_results

In [ ]:
def plot_time_bin_avg(time_bin_power_avg,time_bin_power_std_avg,time_bins,z_range,main_title,file_path):
    # Plot average time-bin events for each frequency
    
    import matplotlib.cm as cm
    from pylab import locator_params
    colors = cm.rainbow(np.linspace(0, 1, len(freqs)))

    time_bin_label = range(1,time_bins+1)
    frequency_labels = ('3 Hz', '5 Hz', '10 Hz','17 Hz','31 Hz','55 Hz','100 Hz','180 Hz')

    fig, axs = plt.subplots(3, 3, sharey = True,figsize=(6,4)) # Plots only 8 frequncy bands
    fig.tight_layout()
    plt.subplots_adjust(top=0.88,bottom=.1,left=0.1,wspace=0.4,hspace=0.5)
    y_data = np.transpose(time_bin_power_avg[:,:]) # Simplify plot command below # (electrode pairs, frequency, time bins)
    y_std = np.transpose(time_bin_power_std_avg[:,:])

    for i_plot in range(0,len(freqs)):
        plt.subplot(3,3,i_plot+1)
        plt.plot(time_bin_label,y_data[:,i_plot],color=colors[i_plot,:], label=frequency_labels[i_plot])
        plt.fill_between(time_bin_label,y_data[:,i_plot]-y_std[:,i_plot],y_data[:,i_plot]+y_std[:,i_plot],alpha=.3,facecolor=colors[i_plot,:])
        plt.axhline(0,color='black',linestyle='--') # Plot zero line
        title(frequency_labels[i_plot],fontsize=10)
        axis([1,time_bins, -(z_range), z_range])
        locator_params(axis='y',nbins=4) # Sets the number of tick marks on the y axis

        # Combination subplot
        plt.subplot(3,3,9)
        plt.plot(time_bin_label,y_data[:,i_plot],color=colors[i_plot,:], label=frequency_labels[i_plot])
        plt.fill_between(time_bin_label,y_data[:,i_plot]-y_std[:,i_plot],y_data[:,i_plot]+y_std[:,i_plot],alpha=.3,facecolor=colors[i_plot,:])
        plt.axhline(0,color='black',linestyle='--') # Plot zero line
        title('All Frequencies',fontsize=10)
        axis([1,time_bins, -(z_range),z_range])
        locator_params(axis='y',nbins=4)
        
    # plt.legend(loc=4,prop={'size':9}) # For prior plotting method

    # Entire figure labels
    fig.text(0.5, 0.0, 'Time Bins', ha='center',size=12)
    fig.text(0.0, 0.5, 'z Score', va='center', rotation='vertical',size=12)
    fig.suptitle(main_title, fontsize=16)

#     plt.savefig(file_path) # UNCOMMENT IF WANT IMAGE SAVED
    
    plt.show()

#     savefig('time_bin_analysis.pdf') # Save figure

In [ ]:
def timefreq_average(data_z_pow_wavelet,base_events,elec_index,time_length,event_type,data_locked):
# Averaging over events for a subset of electrdoes for either stimulus-locked or response-locked
    # time_length = 4 # given in seconds
    # data_locked: stimulus-locked = STIM, response-locked = RESP
    # event_type = FAST or SLOW
    
    samp_freq = int(array(data_z_pow_wavelet[0].samplerate))
    
    # Only including correct events - COMMENTED because correct events should already be fitlered
#     events_boolean_cor = base_events['iscorrect'] == 1
#     base_events = base_events[events_boolean_cor]
    
    # Fast or Slow Events Separation
    if event_type == 'FAST':
        # Finding all of the fast events
        events_boolean = base_events['rectime'] < median(base_events['rectime']) # Changed to median from mean
    elif event_type == 'SLOW':
        # Finding all of the slow events
        events_boolean = base_events['rectime'] > median(base_events['rectime'])
        
    # Filtering events
    rectimes_events = base_events['rectime'][events_boolean]
    events_boolean = rectimes_events > (time_length*1000) # Filtering to only fast events that are greater than a threshold
    events_index = np.where(events_boolean)[0]
        
    data_z_pow_wavelet_filt = [data_z_pow_wavelet[i] for i in events_index] # Filters the array to only fast or slow events and above threshold
    
    # Defining the time period for time-frequency plot
    # Structure = [Events, Electrodes, Frequency, Time]
    data_period = np.zeros((len(events_index),len(elec_index),len(data_z_pow_wavelet[0].frequency),(time_length*samp_freq))) # Defining matrix of events, frequency, time

    # Specifying either stimulus or response-locked analysis
    if data_locked == 'STIM':
        # Looping through events to create array of data
        for i_event in range(0,len(data_z_pow_wavelet_filt)):
            data_array_temp = array(data_z_pow_wavelet_filt[i_event]) # Adding data to a matrix from one electrode over multiple events
            data_period[i_event,:,:,:] = data_array_temp[elec_index,:,0:(time_length*samp_freq)] # [Events, Electrodes, Frequency, Time]
    elif data_locked == 'RESP':
        # Looping through events to create array of data
        for i_event in range(0,len(data_z_pow_wavelet_filt)):
            data_array_temp = array(data_z_pow_wavelet_filt[i_event]) # Adding data to a matrix from one electrode over multiple events
            data_period[i_event,:,:,:] = data_array_temp[elec_index,:,-(time_length*samp_freq):] # [Events, Electrodes, Frequency, Time]
            
    data_elec_mean = mean(data_period,axis=1) # Mean over the electrodes
    mean_time_freq_data = mean(data_elec_mean,axis=0) # Mean over the events
    
    return mean_time_freq_data, data_elec_mean

In [ ]:
def plot_timefreq(timefreq_data,freqs,time_period,main_title,data_locked,colorbar_max,colorbar_label):
    # Setting the time and frequency values for plot
    #freqs = np.logspace(np.log10(3), np.log10(180), 8)
#     colorbar_max = .15
    xtick_labels = range(-3,0)


    fig = figure()
    ax = fig.add_subplot(111) # equivalent to (1,1,1)
    
    if data_locked == 'STIM':
        cax = matshow(timefreq_data, aspect='auto', fignum=0, origin ='lower', interpolation='nearest',extent=[0,time_period,0,8],vmin=-colorbar_max,vmax=colorbar_max,cmap=cm.seismic)
    elif data_locked == 'RESP': # Different time labeling
        cax = matshow(timefreq_data, aspect='auto', fignum=0, origin ='lower', interpolation='nearest',extent=[-time_period,0,0,8],vmin=-colorbar_max,vmax=colorbar_max,cmap=cm.seismic)
    
    # Color bar attributes
    cbar = fig.colorbar(cax)
    cbar.set_label(colorbar_label, rotation=270,fontsize=12,labelpad=15)
    
    tick_freq = np.round(freqs) # Rounding frequency to whole numbers for labels

    # Y axis 
    T=arange(len(tick_freq))+0.5 # Moves the major ticks a half step up
    ax.set_yticks(T)
    ax.set_yticklabels(tick_freq)
    ylabel('Frequency (Hz)',fontsize=12)

    # X axis
    ax.xaxis.set_ticks_position('bottom')
    xlabel('Time (s)',fontsize=12)
    
    title(main_title,fontsize=16)

In [ ]:
def accuracy_calculator(data):
    # Calculating Accuracy, Input: event structure, Output: Accuracy in double format from 0 to 1 
    i_math_probs_correct = (data['iscorrect'] == 1)

    math_probs_correct = data[i_math_probs_correct]

    math_accuracy = float(math_probs_correct.shape[0])/float(data.shape[0])
    
    return math_accuracy

In [1]:
def plot_block_plot(data,cbar_label,range_vmin,range_vmax):
    fig = figure(figsize=(8,6))

    for x_value in range(1,10):
        # Separating accuracy by x value
        x_value_plot = data[(x_value-1)*81:(x_value-1)*81+81]
        x_value_plot_reshape = np.reshape(x_value_plot,[9,9])

        # Plot the combination results for accuracy
        axes = plt.subplot(3,3,x_value)
        im = pcolor(flipud(x_value_plot_reshape),vmin=range_vmin,vmax=range_vmax,cmap='jet') # jet_r for colorbar reversed or use seismic for r/b
        plt.title('%s+y+z'%(x_value))
        plt.tight_layout()
        axes.set_yticklabels(list('987654321'))
        axes.set_xticklabels(list('123456789'))
        
        # Setting the x and y ticks in the middle of the box
        axes.set_xticks(np.arange(x_value_plot_reshape.shape[0])+0.5)
        axes.set_yticks(np.arange(x_value_plot_reshape.shape[1])+0.5)


    plt.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85,.15,.05,.7]) # Adds an area to place the color bar
    cbar = fig.colorbar(im,cax=cbar_ax)
    cbar.set_label(cbar_label,rotation=270,labelpad=20,fontsize=15)

In [ ]:
def plot_block_plot_upd(data,cbar_label,range_vmin,range_vmax):
    # 12/2017: Changed for manuscript figure - adjusted labels and font sizes
    fig = figure(figsize=(8,6))

    for x_value in range(1,10):
        # Separating accuracy by x value
        x_value_plot = data[(x_value-1)*81:(x_value-1)*81+81]
        x_value_plot_reshape = np.reshape(x_value_plot,[9,9])

        # Plot the combination results for accuracy
        axes = plt.subplot(3,3,x_value)
        im = pcolor(flipud(x_value_plot_reshape),vmin=range_vmin,vmax=range_vmax,cmap='jet') # jet_r for colorbar reversed or use seismic for r/b
        plt.title('%s+y+z'%(x_value),fontsize=18)
        plt.tight_layout(pad=0.5)
        
        # Adjusts labels for each plot
        if (x_value == 1) or (x_value == 4):
#             axes.set_yticklabels(list(['','8','','6','','4','','2','']),fontsize=16)
            axes.set_yticklabels(list(['9','','7','','5','','3','','1']),fontsize=18)
            axes.set_xticklabels(list(['','','','','','','','','']))
        elif (x_value == 8) or (x_value == 9):
#             axes.set_xticklabels(list(['','2','','4','','6','','8','']),fontsize=16)
            axes.set_xticklabels(list(['1','','3','','5','','7','','9']),fontsize=18)
            axes.set_yticklabels(list(['','','','','','','','','']))
        elif x_value == 7:
#             axes.set_xticklabels(list(['','2','','4','','6','','8','']),fontsize=16)
#             axes.set_yticklabels(list(['','8','','6','','4','','2','']),fontsize=16)
            axes.set_xticklabels(list(['1','','3','','5','','7','','9']),fontsize=18)
            axes.set_yticklabels(list(['9','','7','','5','','3','','1']),fontsize=18)
        else: 
            axes.set_yticklabels(list(['','','','','','','','','']))
            axes.set_xticklabels(list(['','','','','','','','','']))

        # Setting the x and y ticks in the middle of the box
        axes.set_xticks(np.arange(x_value_plot_reshape.shape[0])+0.5)
        axes.set_yticks(np.arange(x_value_plot_reshape.shape[1])+0.5)
    
    plt.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.82,.15,.05,.7]) # Adds an area to place the color bar
    cbar = fig.colorbar(im,cax=cbar_ax)
    cbar.set_label(cbar_label,rotation=270,labelpad=20,fontsize=18)
    cbar.ax.tick_params(labelsize=16)

In [1]:
def fdr_orig(pvalue_list,q_value):
### FDR Calculation
# pvalue_list = single column list of p-values
# q_value = desired significance level

    # q_value = 0.05 # Function input

    p = np.sort(np.squeeze(pvalue_list))
    V = np.array(len(pvalue_list_sort)).astype(float)
    I = np.array(range(0,V)).astype(float)

    cVN = sum(1./(I+1))

    pID = p[max(np.where(p<=(I/V*q_value))[0])]

    pN = p[max(np.where(p<=(I/V*q_value/cVN))[0])]

    return pID, pN

In [ ]:
def fdr(pvalue_list,q_value):
### FDR Calculation
# pvalue_list = single column list of p-values
# q_value = desired significance level

    # q_value = 0.05 # Function input

    p = np.sort(np.squeeze(pvalue_list))
    V = np.array(len(pvalue_list)).astype(float)
    I = np.array(range(1,(V+1).astype(int))).astype(float) # Changed range values

    cVN = sum(1./(I))

    pID = p[max(np.where(p<=(I/V*q_value))[0])]

    try: # Added try clause since it could give error
        pN = p[max(np.where(p<=(I/V*q_value/cVN))[0])]
    except:
        pN = 0 # there will be no significant points

    return pID, pN

In [ ]:
def remove_NaN_subs(timefreq_ROI):
    ## Removing subjects without time-frequency data - for timing analysis
    # Input: list of ROIs with array of [subjects,frequency,time]
    timefreq_ROI_del = []
    
    for index, ROI in enumerate(ROI_list):
        timefreq_ROI_del.append(np.delete(timefreq_ROI[index],np.unique(np.where(isnan(timefreq_ROI[index]))[0]),0))
        
    return timefreq_ROI_del